# User Guide

:::{toctree}
:titlesonly:
:maxdepth: 2
:hidden:

api
:::

xarray-jsonschema provides a simple class-based API for defining schemas and validating xarray objects.

All schema share the same API, with the {py:meth}`~.XarraySchema.validate()` method doing most of the work.

Let's start with an example {py:class}`DataArray` that we can validate.

In [12]:
import numpy as np
import xarray as xr

from xarray_jsonschema import DataArraySchema

da = xr.DataArray(
    np.ones((4, 10), dtype='i4'),
    dims=('x', 't'),
    name='foo',
    attrs={'spam': 'ham', 'eggs': 42},
)

We can use {py:class}`DataArraySchema` to create a schema for this `DataArray`:

In [13]:
schema = DataArraySchema(
    dtype=np.int32,
    dims=['x', 't'],  # Can be any `Sequence`
    name='foo',
    shape=(4, None),  # ``None`` is used as a wildcard
    attrs={'spam': str, 'eggs': int},
)

To validate the `DataArray` we simply call the `.validate()` method:

In [14]:
schema.validate(da)

Or alternatively, we can call the schema directly:

In [ ]:
schema(da)

If validation is successful, nothing happens. Otherwise, a {py:class}`~.ValidationError` is raised:

In [16]:
from xarray_jsonschema import ValidationError

da2 = da.astype('int64')

try:
    schema.validate(da2)
except ValidationError as error:
    print(error.message)

'int32' was expected


## Component Schemas

Most of the key components of the `xarray` [data model](https://docs.xarray.dev/en/stable/user-guide/index.html) are implemented as standalone classes. These classes allow you to specify more complex constraints.

In [ ]:
from xarray_jsonschema import (
    AttrSchema,
    AttrsSchema,
    DimsSchema,
    DTypeSchema,
    NameSchema,
    ShapeSchema,
    SizeSchema,
)

name_schema = NameSchema('^fo+', regex=True)
dims_schema = DimsSchema(contains=NameSchema(['time', 't']))
shape_schema = ShapeSchema(
    (None, SizeSchema(maximum=10))
)  # ``None`` is used as a wildcard
dtype_schema = DTypeSchema('int32')
attrs_schema = AttrsSchema(
    {
        '^s[a-z]+$': AttrSchema(str, regex=True),
    },
    strict=False,  # Allow un-evaluated items
)


The component schema can be used to validate:

In [23]:
name_schema.validate(da)

But typically they are composed with a container schema like {py:class}`DataArraySchema`:

In [27]:
schema2 = DataArraySchema(
    name=name_schema,
    dims=dims_schema,
    shape=shape_schema,
    dtype=dtype_schema,
    attrs=attrs_schema,
)
schema2.validate(da)

## JSON Schema

Use the {py:attr}`~.XarraySchema.json` property to view the JSON Schema representation:

In [28]:
schema.json

{'type': 'object',
 'properties': {'dims': {'type': 'array',
   'items': False,
   'prefixItems': [{'const': 'x'}, {'const': 't'}],
   'minItems': 2},
  'attrs': {'type': 'object',
   'properties': {'spam': {'type': 'string'}, 'eggs': {'type': 'integer'}},
   'additionalProperties': True,
   'required': ['spam', 'eggs']},
  'dtype': {'const': 'int32'},
  'shape': {'type': 'array',
   'items': False,
   'prefixItems': [{'const': 4}, {'type': 'integer'}],
   'minItems': 2},
  'name': {'const': 'foo'}}}

Or call the {py:meth}`~.XarraySchema.dumps` method to serialize the schema to a JSON formatted string:

In [29]:
print(schema.dumps(indent=2))

{
  "type": "object",
  "properties": {
    "dims": {
      "type": "array",
      "items": false,
      "prefixItems": [
        {
          "const": "x"
        },
        {
          "const": "t"
        }
      ],
      "minItems": 2
    },
    "attrs": {
      "type": "object",
      "properties": {
        "spam": {
          "type": "string"
        },
        "eggs": {
          "type": "integer"
        }
      },
      "additionalProperties": true,
      "required": [
        "spam",
        "eggs"
      ]
    },
    "dtype": {
      "const": "int32"
    },
    "shape": {
      "type": "array",
      "items": false,
      "prefixItems": [
        {
          "const": 4
        },
        {
          "type": "integer"
        }
      ],
      "minItems": 2
    },
    "name": {
      "const": "foo"
    }
  }
}
